## Custom gates

This plugin provides a set of custom gates that you can use in order to express your quantum circuits in MonarQ's native gate set. 

Here are the operations from MonarQ's native gate set : 

In [ ]:
# this section is for initializing the document's code


# import custom gates
import pennylane_snowflurry.processing.custom_gates as custom

# import pennylane-specific functions
import pennylane as qml
import numpy as np
from pennylane.operation import Operation

dev = qml.device("default.qubit")

def print_matrix(op : Operation):
    print(op.name + " : ")
    print(f"{op.matrix()}\n")

### one qubit gates

In [13]:
# a pi rad rotation around the X axis (has the effect of an amplitude flip)
x = qml.PauliX(0) 
print_matrix(x)

# a pi rad rotation around the Z axis (has the effect of a phase flip)
z = qml.PauliZ(0)
print_matrix(z)

# a pi rad rotation around the Y axis (has the effect of combining a X and a Z)
y = qml.PauliY(0) 
print_matrix(y)

# a pi/2 rad rotation around the Z axis
z90 = custom.Z90(0)
print_matrix(z90)

# a -pi/2 rad rotation around the Z axis
zm90 = custom.ZM90(0)
print_matrix(zm90)

# a pi/2 rad rotation around the X axis
x90 = custom.X90(0)
print_matrix(z90)

# a -pi/2 rad rotation around the X axis
xm90 = custom.XM90(0)
print_matrix(zm90)

# a pi/2 rad rotation around the Y axis
y90 = custom.Y90(0)
print_matrix(y90)

# a -pi/2 rad rotation around the Y axis
ym90 = custom.YM90(0)
print_matrix(ym90)

# a pi/4 rad rotation around the Z axis
t = qml.T(0)
print_matrix(t)

# a -pi/4 rad rotation around the Z axis
tdag = custom.TDagger(0)
print_matrix(tdag)

# an arbitrary rotation around the Z axis
p = qml.PhaseShift(2*np.pi/3, 0)
print_matrix(p)

PauliX : 
[[0 1]
 [1 0]]

PauliZ : 
[[ 1  0]
 [ 0 -1]]

PauliY : 
[[ 0.+0.j -0.-1.j]
 [ 0.+1.j  0.+0.j]]

Z90 : 
[[0.70710678-0.70710678j 0.        +0.j        ]
 [0.        +0.j         0.70710678+0.70710678j]]

ZM90 : 
[[0.70710678+0.70710678j 0.        +0.j        ]
 [0.        +0.j         0.70710678-0.70710678j]]

Z90 : 
[[0.70710678-0.70710678j 0.        +0.j        ]
 [0.        +0.j         0.70710678+0.70710678j]]

ZM90 : 
[[0.70710678+0.70710678j 0.        +0.j        ]
 [0.        +0.j         0.70710678-0.70710678j]]

Y90 : 
[[ 0.70710678+0.j -0.70710678-0.j]
 [ 0.70710678+0.j  0.70710678+0.j]]

YM90 : 
[[ 0.70710678+0.j  0.70710678-0.j]
 [-0.70710678+0.j  0.70710678+0.j]]

T : 
[[1.        +0.j         0.        +0.j        ]
 [0.        +0.j         0.70710678+0.70710678j]]

TDagger : 
[[1.        +0.j         0.        +0.j        ]
 [0.        +0.j         0.70710678-0.70710678j]]

PhaseShift : 
[[ 1. +0.j         0. +0.j       ]
 [ 0. +0.j        -0.5+0.8660254j]]



### 2 qubits gates

In [14]:
# if control (leftmost) qubit is set to 1, then target (rightmost) qubit will be applied a Z operation.
cz = qml.CZ([0, 1])
print_matrix(cz)

CZ : 
[[ 1  0  0  0]
 [ 0  1  0  0]
 [ 0  0  1  0]
 [ 0  0  0 -1]]



Here is an example of how you could use those operations in a circuit

In [ ]:
from pennylane_snowflurry.processing.config import EmptyConfig
from pennylane_snowflurry.API.client import MonarqClient

client = MonarqClient("your host", "your user", "your access token", "your project name")

# say you want to transpile your circuit by yourself
config = EmptyConfig() 

dev = qml.device("monarq.default", shots=1000, client = client, processing_config = config)

# This would be equivalent to a bell state (Hadamard(0), CNOT([0, 1]))
# If we observe MonarQ's connectivity, we find that qubits 0 and 4 are adjacent
# If we search online a bit, we find that Hadamard(i) is equivalent to RZ(pi/2, i) - RX(pi/2, i) - RZ(pi/2, i)
# We also find that CNOT([i, j]) is equivalent to H(j) - CZ([i, j]) - H(j)
@qml.qnode(dev)
def circuit():
    # native decomposition of a hadamard gate
    custom.Z90(0)
    custom.X90(0)
    custom.Z90(0)
    
    # native decomposition of a CNOT gate
    custom.Z90(4)
    custom.X90(4)
    custom.Z90(4)
    
    qml.CZ([0, 4])
    
    custom.Z90(4)
    custom.X90(4)
    custom.Z90(4)
    
    return qml.coutns(wires=[0, 4])

results = circuit()
print(circuit)

Even though there may be scenarios where it is beneficial to bypass the transpiling capabilites of the plugin, it is strongly recommended to stick to the default configuration. You don't have to pass any configuration to the device to use the default preset.

In [ ]:
client = MonarqClient("your host", "your user", "your access token", "your project name")

dev = qml.device("monarq.default", shots=1000, client = client)